We used Google drive to save our raw and cleaned dataset.This step is optional.If you are not using Google drive, please use the path with "./data" directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import Library

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import gc
import matplotlib.pyplot as plt

Load encoded dataset

In [ ]:
df = pd.read_csv('./drive/MyDrive/train_encoded.csv')
# df = pd.read_csv('./data/train_encoded.csv')

Split the dataset into training and testing.

In [ ]:
target_column = "HasDetections"
data_id = 'MachineIdentifier'
X = df.drop(target_column, axis=1).drop(data_id, axis=1)
y = df[target_column]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Delete the loaded whole dataset to save memory.

In [ ]:
del df
gc.collect()

30

Create the LightGBM data containers

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

Define the parameters

In [ ]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'num_iterations': 100
}

Train the model

In [ ]:
gbm = lgb.train(params, train_data, valid_sets=[test_data])

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 892084, number of negative: 892212
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.016717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5313
[LightGBM] [Info] Number of data points in the train set: 1784296, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499964 -> initscore=-0.000143
[LightGBM] [Info] Start training from score -0.000143


Predict on the test set

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

Convert probabilities to binary output

In [ ]:
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]

Evaluate the model

In [ ]:
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6446673765622373
